# 1. Prostory jmen a obor platnosti proměnných
V tomto notebooku se zaměříme na to, jak Python vyhledává názvy proměnných a proč stejný název může v různých blocích znamenat něco jiného.

V dalším notebooku na to navážeme třídami.


## 1.1 Typy prostorů jmen a pravidlo LEGB
Prostor jmen (`namespace`) si můžete představit jako mapování názvů na objekty.

Python při hledání názvu postupuje v pořadí LEGB:
- lokální (`local`) - aktuální funkce
- ne-lokální (`enclosing`/`nonlocal`) - obalující funkce
- globální (`global`) - modul
- vestavěný (`builtins`) - vestavěná jména jazyka

Když název nenajde ani v jednom z nich, vyvolá `NameError`.
Globální i lokální prostor jsou slovníky; vestavěný prostor je dostupný přes modul `builtins`.


In [ ]:
nejaka_promenna = 1
print(type(globals()))
print(globals())


In [ ]:
nejaka_promenna = 1

def funkce():
    nova_promenna_uvnitr_funkce = 1
    nejaka_promenna = 2
    print(type(locals()))
    print(locals())
    
funkce()

In [ ]:
print(type(__builtins__))
print(dir(__builtins__))

In [ ]:
# pokud jméno není v žádném prostoru, záměrně vyvoláme NameError
print(nedefinovana_promenna)
nedefinovana_promenna = 1


In [ ]:
# pokud vyrobíme novou proměnnou zde, bude v globálním prostoru
moje_promenna = 42
print(id(moje_promenna))
print(id(globals()['moje_promenna']))


In [ ]:
# ukázka globálního prostoru
moje_promenna = 42


def funkce():
    print(moje_promenna)


funkce()


In [ ]:
# ukázka lokálního prostoru
moje_promenna = 42


def funkce():
    moje_promenna = 43
    print(moje_promenna)


funkce()


In [ ]:
# ukázka nelokálního prostoru
moje_promenna = 42


def funkce():
    moje_promenna = 43

    def funkce2():
        print(moje_promenna)

    funkce2()


funkce()


## 1.2 Lokální prostor a `UnboundLocalError`
Lokální prostor vzniká při volání funkce, ne při její definici.

Pokud funkce do proměnné přiřazuje, Python ji považuje za lokální v celé funkci. Čtení před přiřazením pak skončí `UnboundLocalError`.


In [ ]:
# tato ukázka záměrně vyvolá UnboundLocalError
moje_promenna = 42


def funkce():
    print(moje_promenna)
    moje_promenna = 43
    print(moje_promenna)


funkce()


## 1.3 Práce s globálním prostorem
Z funkce můžeme měnit obsah mutovatelného globálního objektu (např. listu), aniž bychom použili `global`.

Jakmile ale chceme jméno přesměrovat na nový objekt, Python ho bez `global` bere jako lokální.


In [ ]:
muj_list = [1, 2, 3]


def funkce():
    print(muj_list)
    muj_list[0] = 42
    muj_list.append(4)


funkce()
print(muj_list)


Pro přepsání globální vazby je potřeba `global`.

V praxi je lepší se tomu spíš vyhýbat, protože to zhoršuje čitelnost a testovatelnost kódu.


In [ ]:
muj_list = [1, 2, 3]

print(id(muj_list))


def funkce():
    global muj_list
    muj_list = [42, 2, 3]


funkce()

print(id(muj_list))
print(muj_list)


## 1.4 `nonlocal` v zanořených funkcích
Stejný princip platí pro obalující funkci. Pokud chceme ve vnořené funkci přesměrovat jméno z obalujícího scope, použijeme `nonlocal`.


In [ ]:
# ukázka nonlocal
def funkce():
    muj_list = [1, 2, 3]

    def funkce2():
        nonlocal muj_list
        muj_list[0] = 42
        muj_list.append(4)

    funkce2()
    print(muj_list)


funkce()


## 1.5 Obor platnosti v praxi
Následující ukázka shrnuje, že stejný název může existovat současně v různých scopech bez vzájemného přepisování.


In [ ]:
promenna = 42


def funkce():
    promenna = 43
    print("1:", promenna)

    def funkce2():
        promenna = 44
        print("2:", promenna)

    funkce2()
    print("3:", promenna)


print("4:", promenna)
funkce()
print("5:", promenna)


## 1.6 Výchozí hodnoty parametrů
Výchozí hodnoty parametrů se vyhodnotí jednou při definici funkce.

U mutovatelných typů (např. `list`) se proto změny mezi voláními sdílejí. Přiřazení na lokální proměnnou uvnitř funkce ale původní default nepřepíše.


In [ ]:
def funkce(muj_list = []):
    muj_list.append(42)
    print(muj_list)
    muj_list = []
    
funkce()
funkce()
funkce([])
funkce()
funkce()


Na aktuální výchozí hodnoty parametrů se lze podívat přes atribut `__defaults__`.


In [ ]:
def funkce(muj_list=[], muj_string=""):
    muj_list.append(42)
    muj_string += "42"
    print(muj_list)
    print(muj_string)


print(funkce.__defaults__)
funkce()
print(funkce.__defaults__)
funkce()
print(funkce.__defaults__)
funkce()
print(funkce.__defaults__)
funkce()
print(funkce.__defaults__)